In [1]:
import sys
print(sys.executable)


c:\Users\harik\Downloads\SkillDzire\ML\HousePrice\myenv\Scripts\python.exe


In [2]:
from xgboost import XGBRegressor
print("XGBoost imported successfully!")


XGBoost imported successfully!


In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from xgboost import XGBRegressor


In [4]:
#pip install xgboost

In [5]:
# Read csv files. And keep a copy of the original dataframes. tran_df and test_df will be used throughout this notebook for preprocessing and scaling.we need original test dataframe in the end 
or_train_df = pd.read_csv('train.csv')
or_test_df = pd.read_csv('test.csv')

train_df = or_train_df
test_df = or_test_df

In [6]:
# Dropping 'Id' column as it has no use in training
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

In [7]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1460, 80)
Shape of test set: (1459, 79)


In [8]:
train_df.head(), test_df.head()

(   MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
 0          60       RL         65.0     8450   Pave   NaN      Reg   
 1          20       RL         80.0     9600   Pave   NaN      Reg   
 2          60       RL         68.0    11250   Pave   NaN      IR1   
 3          70       RL         60.0     9550   Pave   NaN      IR1   
 4          60       RL         84.0    14260   Pave   NaN      IR1   
 
   LandContour Utilities LotConfig  ... PoolArea PoolQC Fence MiscFeature  \
 0         Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
 1         Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   
 2         Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
 3         Lvl    AllPub    Corner  ...        0    NaN   NaN         NaN   
 4         Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   
 
   MiscVal MoSold  YrSold  SaleType  SaleCondition  SalePrice  
 0       0      2    2008        WD         

## Replace NaNs
**While replacing we have to make sure to replace it with '0' (character) in case of a string column and 0 (number) if otherwise**

In [9]:
str_columns = []
num_columns = []

def convert_to_str(df):
    for col in df.columns:
        if (df[col].dtype == np.int64 or df[col].dtype == np.float64):
            df[col] = df[col].fillna(method='ffill')
            if col not in num_columns:
                num_columns.append(col)
        else:
            df[col] = df[col].fillna(method='ffill')
            if col not in str_columns:
                str_columns.append(col)
    return df

train_df = convert_to_str(train_df)
test_df = convert_to_str(test_df)

C:\Users\harik\AppData\Local\Temp\ipykernel_10520\656724134.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill')
C:\Users\harik\AppData\Local\Temp\ipykernel_10520\656724134.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill')


In [10]:
train_df['type'] = 'train'
test_df['type'] = 'test'

# Add a dummy SalePrice column to test dataframe to make number of columns equal 
test_df['SalePrice'] = train_df['SalePrice'].iloc[:201]

df = train_df._append(test_df, ignore_index=True)

In [11]:
for col in str_columns:
    one_hot = pd.get_dummies(df[col])

    replace_cols = {}
    for one_col in one_hot.columns:
        replace_cols[one_col] = f"{col}_{one_col}"
    one_hot = one_hot.rename(columns=replace_cols)

    df = df.drop(col, axis = 1)
    df = df.join(one_hot)

In [12]:
train_df = df[df['type'] == 'train']
test_df = df[df['type'] == 'test']

train_df = train_df.drop(['type'], axis=1)
test_df = test_df.drop(['type'], axis=1)

test_df = test_df.reset_index(drop=True)

In [13]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1460, 288)
Shape of test set: (1459, 288)


## Standardization of values

In [14]:
scaler = StandardScaler()
scaler.fit(train_df[num_columns])

StandardScaler()

In [15]:
train_df[num_columns] = scaler.transform(train_df[num_columns])
test_df[num_columns] = scaler.transform(test_df[num_columns])

In [16]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1460, 288)
Shape of test set: (1459, 288)


In [17]:
test_df = test_df.drop(['SalePrice'], axis=1)

In [18]:
train_labels = train_df['SalePrice']
train_data = train_df.drop(['SalePrice'], axis=1)

## XGBoost regressor model 

In [19]:
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [20]:
model.fit(train_data, train_labels)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [21]:
predictions = model.predict(test_df)

In [22]:
import joblib
joblib.dump(model, "house_price_model.pkl")


['house_price_model.pkl']

In [23]:
# After preprocessing and one-hot encoding
joblib.dump(train_data.columns.tolist(), 'model_columns.pkl')


['model_columns.pkl']

In [24]:
test_df['SalePrice'] = predictions
test_df[num_columns] = scaler.inverse_transform(test_df[num_columns])
test_df = pd.DataFrame(test_df, columns=train_df.columns)

In [25]:
# Create results dataframe
results = pd.DataFrame()
results['Id'] = or_test_df['Id']
results['SalePrice'] = test_df['SalePrice']

In [26]:
results.head()

,Id,SalePrice
0,1461,125349.814747
1,1462,162221.499829
2,1463,195578.424229
3,1464,190139.793463
4,1465,192553.068123


In [27]:
results.to_csv('submissions.csv', index=False)